# Filtering 

Let's start from all the transiting exoplanets in the NASA Exoplanet Archive, and then filter down into a `Population` of planets we might want to consider observing. 

In [ ]:
import exoatlas as ea 
ea.version()

## Download Exoplanet Data

Let's download a recent list of transiting exoplanets from the NASA Exoplanet Archive. We should, as always, be careful not to completely trust every detail of this database. For any particular planet, there's a possibility that details might be wrong or missing so for anything super crucial, it would be very wise to double check. 

In [ ]:
e = ea.TransitingExoplanets()


Throughout, let's use `physical_summary` to visualize how the various filtering cuts remove planets from our observing sample. 

In [ ]:
ea.physical_summary(e);

## Define Filtering Criteria

Let's make some cuts to this sample in order to select systems that might be worth trying to observe at the telescope. 


### Is it big enough? 

To ensure that a planet has a hydrogen-dominated atmosphere (which makes interpreting a transmission spectrum a lot easier), we could place a cut on the planet's radius. ["Most 1.6 Earth-radius planets are not rocky"](https://ui.adsabs.harvard.edu/abs/2015ApJ...801...41R/abstract), so let's try cutting there. 

In [ ]:
import astropy.units as u 

is_big = e.radius > 1.6*u.Rearth  
filtered = e[is_big]
filtered

In [ ]:
ea.physical_summary(filtered);

### Is the duration short enough? 

An ideal transit observation would include at least some baseline before the start and after the end of the transit itself. When observing from a ground-based telescope, practically that means it's really hard to observe (complete) transits with durations longer than about a few hours. 

In [ ]:
is_short = e.transit_duration < 3*u.hour
filtered = e[is_short]
filtered

In [ ]:
ea.physical_summary(filtered);

### Is transmission spectroscopy possible? 

The best precision we could possibly acheive on measuring a transit depth is set by the number of photons that we can gather with our telescope. If the signal we're hoping to detect for transmission spectroscopy (see [Observing](observing.ipynb)) is smaller than that expected precision, it's basically hopeless that we'll see anything. So, let's at least limit our sample to targets with a reasonably high predicted signal-to-noise ratio.

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 

signal = e.transmission_signal()
wavelength = 0.7*u.micron
dt = 1*u.hour
R = 10
noise = e.depth_uncertainty(telescope='APO', 
                            wavelength=wavelength, 
                            dt=dt, 
                            R=R)
snr = signal/noise 

plt.scatter(noise, signal, c=snr, vmin=1, vmax=10)
plt.xscale('log'); plt.yscale('log')
plt.axis('scaled')
plt.xlabel('')
plt.ylim(1e-6, 1e-2); plt.xlim(1e-6, 1e-2)
plt.plot([1e-6, 1e-2], [1e-6, 1e-2])
plt.colorbar(label='S/N')
plt.xlabel(f'Predicted Photon Noise\n($\lambda=${wavelength}, dt={dt}, R={R})')
plt.ylabel('Possible Transmission\nSignal Size ($2HR_p/R_{\star}^2$)');

In [ ]:
is_snr = snr > 5
filtered = e[is_snr]
filtered

In [ ]:
ea.physical_summary(filtered);

In [ ]:
filtered.name

### Is it above the horizon at night? 

If we're observing at a particular time of year from a particular observatory, only some targets will be high enough in the sky to be observable at night. Let's place a coarse cut on right ascensions and declinations, so we don't waste time considering planets that are beneath the Earth when we're trying to observe. 

In [ ]:
from astroplan import Observer
from astropy.time import Time 

date_string = '2024-09-21'
observatory_string = "APO"
date = Time(date_string)
observatory = Observer.at_site(observatory_string, timezone="US/Hawaii")

In [ ]:
local_midnight = observatory.midnight(date)
sidereal_time_at_midnight = observatory.local_sidereal_time(local_midnight)
ra_at_midnight = sidereal_time_at_midnight
ra_at_midnight

In [ ]:
declination_at_zenith = observatory.latitude
declination_at_zenith

In [ ]:
is_dec = np.abs(e.dec - declination_at_zenith) < 60*u.deg
is_ra = np.abs((e.ra - ra_at_midnight).wrap_at(180*u.deg)) < 6*u.hourangle

filtered = e[is_dec * is_ra]

kw = dict(s=2, alpha=0.25)
plt.scatter(e.ra, e.dec, **kw)
plt.scatter(filtered.ra, filtered.dec, **kw)
plt.xlabel('Right Ascension (degree)')
plt.ylabel('Declination (degree)')
plt.axis('scaled'); plt.ylim(-90, 90); plt.xlim(360, 0)
plt.title(f'{observatory_string} | {date_string}');


In [ ]:
ea.physical_summary(filtered);

## Make Target Sample

After looking at each of them one by one, let's put all of those filtering criteria together.

In [ ]:
targets = e[is_big*is_short*is_snr*is_ra*is_dec]
targets

In [ ]:
ea.physical_summary(targets);

Let's print out the names of the planets that made it through all our cuts.

In [ ]:
print(targets.name)

Let's also make a table that includes more useful information, like positions and transit ephemerides, which we could use as an input to plan an observing run.

In [ ]:
table = targets.create_planning_table()

Let's save that table out to a text file, and also make sure we can load it back in again. 

In [ ]:
table.write(f'planets-to-observe-{observatory_string}-{date_string}.ecsv', overwrite=True)

In [ ]:
from astropy.io.ascii import read 
read(f'planets-to-observe-{observatory_string}-{date_string}.ecsv')